In [2]:
import pandas as pd
import numpy as np
import os

import time
from alibi_detect.od import SpectralResidual

In [16]:
def get_data():
    data_path = '../../../../data/train_data/host'
    dfs = {}
    for file in os.listdir(data_path):
        print('Reading ' + file)
        dfs[file[:-4]] = pd.read_csv(data_path+'/'+file) 
    return dfs

def load_tiago_test(test):
    data_path = '../../../../data/tiago_tests/test'
    df = pd.read_csv(data_path+str(test)+'/kpis.csv')
    return df

def load_test():
    data_path = '../../../../data/test_data/host'
    df = pd.DataFrame(['item_id','name','bomc_id','timestamp','value','cmdb_id'])
    for file in os.listdir(data_path):
        df = pd.concat([df, pd.read_csv(data_path+'/'+file)], ignore_index=True) 
    return df

In [17]:
def get_past(df, curr_time, window_size):
    step_size = 1000*60
    df_new = pd.DataFrame(columns=['timestamp', 'value'])

    for time in [curr_time - step_size*x for x in range(window_size)][::-1]:
        if len(df[df.timestamp == time]) == 0:
            plug_time = np.max(df[df.timestamp < time]['timestamp'])
            row = {}
            row['timestamp'] = time
            if len(df.loc[df.timestamp==plug_time, 'value']) == 0:
                row['value'] = 0
            else: 
                row['value'] = df.loc[df.timestamp==plug_time, 'value'].values[0]
            df_new = df_new.append(row, ignore_index=True)
        else:
            row = {}
            row['timestamp'] = time
            row['value'] = df.loc[df.timestamp==time, 'value'].values[0]
            df_new = df_new.append(row, ignore_index=True)
    return df_new

In [21]:
def get_key_thresh(host):
    df_thresh = pd.read_csv('thresh.data')
    df_thresh = df_thresh[df_thresh.host==host]

    thresh = df_thresh[['name', 'thresh']].set_index('name')
    thresh = thresh.to_dict()['thresh']
    return thresh

def find_anoms(hosts, df):
    start = time.time()
    anoms = []
    for host in hosts:
        start_host = time.time()
        print('*'*60)
        print('Checking ', host)
        thresholds = get_key_thresh(host)
        for kpi in thresholds:
            thresh = thresholds[kpi]
            print('Threshold ', thresh, kpi)
            od = SpectralResidual(
                threshold=thresh,
                window_amp=30,
                window_local=30,
                n_est_points=5,
                n_grad_points=5
            )
            df_hk = df[df.cmdb_id == host][df.name == kpi]
            
            if len(df_hk) == 0:
                print('No valid data to use, skipping')
                continue
                
            data = get_past(df_hk, np.max(df_hk.timestamp.unique()), 30)['value']
            data = np.concatenate([data, data])
            
            if thresh == -1:
                if data[-1] > 0:
                    print("Non-Zero Anomaly!")
                    anoms.append((host, kpi))
            elif np.isnan(thresh):
                print('NaN, Skipping!')
            else:
                outliers = od.predict(data)['data']
                print(outliers['instance_score'])
                if np.mean(data) == 0 or np.std(data) == 0:
                    print("Mean is zero or std is zero")
                    anoms.append((host,kpi))
                if np.sum(outliers['is_outlier'][-31:-30]) > 0:
                    print(outliers['is_outlier'])
                    print("ST Threshold Anomaly!")
                    anoms.append((host,kpi))
        print(host, ' completed in ', time.time() - start_host)
    print('Completed detection of ', len(hosts), 'hosts in ', time.time() - start, 'seconds')
    return anoms


find_anoms(load_test().cmdb_id.unique(), load_test())

************************************************************
Checking  nan
nan  completed in  0.004977226257324219
************************************************************
Checking  os_015
Threshold  -1.0 Sent_errors_packets


<ipython-input-21-2f1c43acab61>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-21-2f1c43acab61>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]


Threshold  6.765946623096813 Processor_load_5_min
[ 6.80806451  0.87993331  0.62882523 -0.76846408  1.12531533 -0.0759314
  0.56504921  0.66187484  0.68617098 -0.42467558  0.34263363 -0.3038197
 -0.88434952 -0.95245295 -0.23419711  0.47552925 -0.72003171 -0.42642499
 -0.62233129 -0.93866515 -0.92162594 -0.85843504 -0.91131843 -0.7081065
  0.17328667  0.55760927  3.86825882  7.4982042   0.64107078  0.70106029
 -0.38569851 -0.33238573 -0.23297705 -0.92168776 -0.27796938 -0.73635499
 -0.36792073 -0.28060287 -0.37357319 -0.77411465 -0.42078654 -0.69221233
 -0.9541386  -0.96993558 -0.21966577  0.52516355 -0.73071076 -0.41767568
 -0.5796002  -0.91277722 -0.91628775 -0.85119895 -0.9430325  -0.65127801
  0.30421865  0.78349756  4.86567413  9.21107032  1.00074429  2.4060099 ]
Threshold  -1.0 Disk_rd_ios


<ipython-input-21-2f1c43acab61>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-21-2f1c43acab61>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]


Threshold  1.1685627874802902 CPU_util_pct
[ 1.06865983  0.44675837  0.16926029  1.272276    0.8435549   1.07040727
  0.03472925 -0.30204951  1.76683203  0.12971441 -0.08277121  0.04415771
 -0.02931504  0.60460602 -0.26567176 -0.07331093 -0.06086862 -0.21438851
  0.35598397 -0.10252708 -0.03378639 -0.29829284 -0.03011717  0.33753076
 -0.20937047 -0.03267298 -0.09275169 -0.0864877  -0.13769943 -0.09665586
  0.11061826 -0.19218614 -0.31553673  0.39591759  0.21483245  0.42231864
 -0.25368206 -0.48099392  1.14779245 -0.07411321 -0.22304258 -0.07950844
 -0.11732     0.50846151 -0.28750196 -0.07449318 -0.02522058 -0.16301776
  0.48449566  0.03877583  0.16890595 -0.10165292  0.28534066  0.81677473
  0.18610693  0.52510158  0.48579456  0.54982322  0.60483985  0.84978407]
Threshold  2.7743162460134143 Disk_io_util


...

Threshold  -1.0 container_fgc
Threshold  -1.0 container_thread_used_pct


<ipython-input-21-2f1c43acab61>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-21-2f1c43acab61>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]


Threshold  0.7931023869961198 container_mem_used
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
Mean is zero or std is zero
docker_001  completed in  1.4352362155914307
Completed detection of  58 hosts in  329.6315498352051 seconds


/Users/baconbaker/anaconda3/envs/ml/lib/python3.8/site-packages/alibi_detect/od/sr.py:96: RuntimeWarning: divide by zero encountered in log
  log_amp = np.log(amp)
/Users/baconbaker/anaconda3/envs/ml/lib/python3.8/site-packages/alibi_detect/od/sr.py:99: RuntimeWarning: invalid value encountered in subtract
  res_amp = log_amp - ma_log_amp
/Users/baconbaker/anaconda3/envs/ml/lib/python3.8/site-packages/alibi_detect/od/sr.py:207: RuntimeWarning: invalid value encountered in greater
  outlier_pred = (iscore > self.threshold).astype(int)


[('os_015', 'Disk_io_util'),
 ('os_015', 'Num_of_processes'),
 ('os_015', 'Incoming_network_traffic'),
 ('os_015', 'Num_of_running_processes'),
 ('os_002', 'Num_of_processes'),
 ('os_002', 'Num_of_running_processes'),
 ('os_002', 'System_block_queue_length'),
 ('os_010', 'Page_pi'),
 ('os_010', 'Disk_rd_kbs'),
 ('os_010', 'Received_packets'),
 ('os_008', 'Outgoing_network_traffic'),
 ('os_008', 'Incoming_network_traffic'),
 ('os_008', 'Disk_wr_ios'),
 ('os_008', 'Sent_queue'),
 ('os_018', 'Disk_io_util'),
 ('os_018', 'Incoming_network_traffic'),
 ('os_018', 'Shared_memory'),
 ('os_018', 'System_block_queue_length'),
 ('os_018', 'Received_packets'),
 ('os_003', 'Disk_io_util'),
 ('os_003', 'Zombie_Process'),
 ('os_003', 'Disk_await'),
 ('os_003', 'System_block_queue_length'),
 ('os_019', 'Outgoing_network_traffic'),
 ('os_019', 'Incoming_network_traffic'),
 ('os_019', 'Zombie_Process'),
 ('os_019', 'Page_pi'),
 ('os_019', 'Disk_rd_kbs'),
 ('os_009', 'Page_pi'),
 ('os_009', 'Disk_await')

In [ ]:
import seaborn as sns

df = load_test(2)
df = df[df.cmdb_id=='os_018'][df.name=='Buffers_used']
sns.lineplot(x='timestamp', y='value',data=df)